In [24]:
import pandas as pd
import json
from difflib import SequenceMatcher

### Part 1: Read a json file of outputs as identified by the classifier

In [39]:
#name = 'roberta_lg_mgn_2'
name = 'elmo_best'
with open("../../predictions/test_set_predictions/enseble_baseline_TRAIN.json") as json_file:
    all_predictions = json.load(json_file)

In [31]:
all_predictionss = dict()

for key in all_predictions.keys():
    if key not in all_predictionss.keys():
        all_predictionss[key] = dict()
    if 'positive_keywords' not in all_predictionss[key].keys():
        all_predictionss[key]['positive_keywords'] = list()
    all_predictionss[key]['positive_keywords'] = all_predictions[key][0]

In [33]:
def split_keyphrases(dict):
    for key in dict.keys():
        if "positive_keywords" in dict[key].keys():
            tmp_pos_keys = []
            #dict[key]["positive_keywords"] = dict[key]["positive_keywords"].split(" ")
            for keyphrase in dict[key]["positive_keywords"]:
                keywords = keyphrase.split(" ")
                tmp_pos_keys.extend(keywords)
            dict[key]["positive_keywords"] = list(set(tmp_pos_keys))
        if "negative_keywords" in dict[key].keys():
            tmp_neg_keys = []
            #dict[key]["negative_keywords"] = dict[key]["negative_keywords"].split(" ")
            for keyphrase in dict[key]["negative_keywords"]:
                keywords = keyphrase.split(" ")
                tmp_neg_keys.extend(keywords)
            dict[key]["negative_keywords"] = list(set(tmp_neg_keys))
    return dict

all_predictions = split_keyphrases(all_predictionss)

In [34]:
with open("../../predictions/train_set_predictions/6_rake_keywords_format_1_TRAIN_fixed.json", "w") as outfile:
    json.dump(all_predictions, outfile)

In [24]:
# Fix completely wrong predictions, where 'NEW LINE' was returned as a movie

for key in all_predictions:
    if 'positive_movies' in all_predictions[key].keys():
        all_predictions[key]["positive_movies"] = [item for item in all_predictions[key]["positive_movies"] if item not in ['/', "The", 'of' "NEW_LINE", '.']]
        if len(all_predictions[key]["positive_movies"]) < 1:
            all_predictions[key].pop('positive_movies')
            
    if 'negative_movies' in all_predictions[key].keys():
        all_predictions[key]["negative_movies"] = [item for item in all_predictions[key]["negative_movies"] if item not in ["/", "The", 'of', "NEW_LINE", "."]]
        if len(all_predictions[key]["negative_movies"]) < 1:
            all_predictions[key].pop('negative_movies')
               
    if 'positive_keywords' in all_predictions[key].keys():
        all_predictions[key]["positive_keywords"] = [item for item in all_predictions[key]["positive_keywords"] if item not in ["/", "NEW_LINE", "."]]
        if len(all_predictions[key]["positive_keywords"]) < 1:
            all_predictions[key].pop('positive_keywords')
            
    if 'negative_keywords' in all_predictions[key].keys():
        all_predictions[key]["negative_keywords"] = [item for item in all_predictions[key]["negative_keywords"] if item not in ["/", "NEW_LINE", "."]]
        if len(all_predictions[key]["negative_keywords"]) < 1:
            all_predictions[key].pop('negativee_keywords')
            
    if 'positive_keywords' in all_predictions[key].keys():
        for i in range(0, len(all_predictions[key]["positive_keywords"])):
            all_predictions[key]["positive_keywords"][i] = all_predictions[key]["positive_keywords"][i].replace("/", "")   
    
    if 'negative_keywords' in all_predictions[key].keys():
        for i in range(0, len(all_predictions[key]["negative_keywords"])):
            all_predictions[key]["negative_keywords"][i] = all_predictions[key]["negative_keywords"][i].replace("/", "")   

### Part 2: Turn the original predictions to format 2 and save them as json

In [40]:
# Change the format of the predictions to fit recommender engine

all_predictions_2 = dict()

for key, value in all_predictions.items():
    if not key in all_predictions_2.keys():
        all_predictions_2[key] = list()
        for k in value.keys():
            if k=='positive_movies':
                for v in value[k]:
                    all_predictions_2[key].append("-mp " + "\"" + v + "\"")
            elif k=='negative_movies':
                for v in value[k]:
                    all_predictions_2[key].append("-mn " + "\"" + v + "\"")
            elif k=='positive_genres':
                for v in value[k]:
                    all_predictions_2[key].append("-gp " + "\"" + v + "\"")
            elif k=='negative_genres':
                for v in value[k]:
                    all_predictions_2[key].append("-gn " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    all_predictions_2[key].append("-amp " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    all_predictions_2[key].append("-amn " + "\"" + v + "\"")
            elif k=='positive_keywords':
                for v in value[k]:
                    all_predictions_2[key].append("-kp " + "\"" + v + "\"")
            elif k=='negative_keywords':
                for v in value[k]:
                    all_predictions_2[key].append("-kn " + "\"" + v + "\"")

In [41]:
with open("../../predictions/test_set_predictions/ensemble_baseline_TRAIN_format_2.json", "w") as outfile:
    json.dump(all_predictions_2, outfile)

In [20]:
# Dictionary of matching movies in several languages 
def create_alt_movie_dict(movies_matched):
    alt_names = dict()
    for index, row in movies_matched.iterrows():
        if row["original_title"] not in alt_names.keys():
            alt_names[row["original_title"]] = list()
        alt_names[row["original_title"]].append(row["alternative"])
    return alt_names

# Similarity function
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [2]:
# Read additional data
imdb_genres = pd.read_csv("genres.csv", sep=';') # IMDB genre list to compare identified genres to
movie_titles = pd.read_csv("movie_titles.csv", sep=';', encoding = 'latin')
movies_matched = pd.read_csv("movies_matched.csv")
submissions = pd.read_csv("test_submissions.csv", sep=';')
submissions = submissions.fillna("")

alt_names = create_alt_movie_dict(movies_matched)

In [30]:
movie = 'Island'
most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:-7]) >= 0.95) or similar('The ' + movie, mov[0:-7]) >= 0.95)]
mov = 'The Lord of the Ringispils: The Fellowship of the Ring (2001)'
mov[0:-7]
similar('Island', 'The Island')
most_similar

['The Island (2005)']

In [31]:
most_similar_alt = []
most_similar_alt_id = []
for k in alt_names.keys():
    for mov in alt_names[k]:
        if similar(movie, mov[0:len(movie)]) >= 0.8:
            most_similar_alt.append(k)
            most_similar_alt.append(mov)
                
most_similar_alt = list(dict.fromkeys(most_similar_alt))
if len(most_similar_alt) > 0:
    most_similar_alt_id = []
    for title in most_similar_alt:
        if title in movie_titles.movie_title.values:
            most_similar_alt_id.append(movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0])
            most_similar_alt_id = list(dict.fromkeys(most_similar_alt_id))

In [32]:
most_similar_alt

['¿Quién puede matar a un niño? (1976)',
 'Island of Death (1980)',
 'Island of the Damned (1978)',
 'Zombi Holocaust (1980)',
 'Island of the Last Zombies (1980)',
 'Zombi 2 (1979)',
 'Island of the Living Dead (1979)',
 'Island of the Flesh-Eaters (1979)',
 'Hrútar (2015)',
 'Islandzka opowiesc (2015)',
 'Ilha das Flores (1989)',
 'Island of Flowers (1989)',
 'Inland Empire (2006)',
 'Inland Empire (2007)',
 "Inland Empire - L'impero della mente (2006)",
 'Inland Empire - Eine Frau in Schwierigkeiten (2007)',
 'Inland Empire: A Woman in Trouble (2006)']

### Part 3: Perform the matching

In [34]:
imdb_predictions = dict()
for key in all_predictions.keys():
    imdb_predictions[key] = dict()
    for category in all_predictions[key].keys():
        if category not in imdb_predictions[key].keys():
            imdb_predictions[key][category] = []
            
        if category in ['positive_keywords', 'negative_keywords']:
            imdb_predictions[key][category] = all_predictions[key][category]
        
        elif category in ['positive_genres', 'negative_genres']:
            for genre in all_predictions[key][category]:
                for genre_imdb in imdb_genres.genrename.values:
                    if genre == genre_imdb:
                        imdb_predictions[key][category].append(genre_imdb)
                    else:
                        if similar(genre, genre_imdb) > 0.70:
                            imdb_predictions[key][category].append(genre_imdb)
            
            all_predictions[key][category] = list(set(all_predictions[key][category]))
            
        elif category in ['positive_actors', 'negative_actors']:
            imdb_predictions[key][category] = all_predictions[key][category]
        
        elif category in ['positive_movies', 'negative_movies']:
            for movie in all_predictions[key][category]:
                most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:-7]) >= 0.9) or similar('The ' + movie, mov[0:-7]) >= 0.9)]
                #most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:len(movie)]) >= 0.8) or similar('The' + movie, mov[0:len(movie)+3]) >= 0.8)]

                if len(most_similar) > 0:
                    most_similar_id = [movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0] for title in most_similar]
                    most_similar_id = list(dict.fromkeys(most_similar_id))
                    #print(most_similar)
                    #print(most_similar_id)
                    imdb_predictions[key][category].extend(most_similar_id)
                else:
                    most_similar_alt = []
                    for k in alt_names.keys():
                        for mov in alt_names[k]:
                            if similar(movie, mov[0:-7]) >= 0.8:
                                most_similar_alt.append(k)
                                most_similar_alt.append(mov)
                
                    most_similar_alt = list(dict.fromkeys(most_similar_alt))
                    if len(most_similar_alt) > 0:
                        most_similar_alt_id = []
                        for title in most_similar_alt:
                            if title in movie_titles.movie_title.values:
                                most_similar_alt_id.append(movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0])
                        most_similar_alt_id = list(dict.fromkeys(most_similar_alt_id))
                        imdb_predictions[key][category].extend(most_similar_alt_id)
            
        #        for movie_database in movie_titles.movie_title
        

In [35]:
all_predictions_matched = imdb_predictions

In [36]:
with open("../../predictions/test_set_predictions/10_spacy_basic_large_movies_matched_format_1.json", "w") as outfile:
    json.dump(imdb_predictions, outfile)

In [37]:
imdb_predictions['5gchx2']

{}

In [38]:
imdb_predictions_2 = dict()
imdb_predictions = all_predictions_matched

for key, value in imdb_predictions.items():
    if not key in imdb_predictions_2.keys():
        imdb_predictions_2[key] = list()
        for k in value.keys():
            if k=='positive_movies':
                for v in value[k]:
                    imdb_predictions_2[key].append("-mp " + "\"" + v + "\"")
            elif k=='negative_movies':
                for v in value[k]:
                    imdb_predictions_2[key].append("-mn " + "\"" + v + "\"")
            elif k=='positive_genres':
                for v in value[k]:
                    imdb_predictions_2[key].append("-gp " + "\"" + v + "\"")
            elif k=='negative_genres':
                for v in value[k]:
                    imdb_predictions_2[key].append("-gn " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    imdb_predictions_2[key].append("-amp " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    imdb_predictions_2[key].append("-amn " + "\"" + v + "\"")
            elif k=='positive_keywords':
                for v in value[k]:
                    imdb_predictions_2[key].append("-kp " + "\"" + v + "\"")
            elif k=='negative_keywords':
                for v in value[k]:
                    imdb_predictions_2[key].append("-kn " + "\"" + v + "\"")

In [39]:
with open("../../predictions/test_set_predictions/10_spacy_basic_large_movies_matched_format_2.json", "w") as outfile:
    json.dump(imdb_predictions_2, outfile)

In [17]:
all_predictions

{'5ciw72': {'positive_keywords': ['Escape',
   'Soylent',
   'Judge',
   'Blade',
   'Element',
   'Runner',
   'Dredd',
   'Green',
   'Fifth']},
 '5ckyug': {'positive_keywords': ['great', 'lengths']},
 '5cpz0k': {'positive_keywords': ['mulholland', 'drive']},
 '5egzb4': {'positive_keywords': ['investigations']},
 '5eqavm': {'positive_keywords': ['films']},
 '5esp73': {'positive_keywords': ['Men', 'Old']},
 '5evio5': {'positive_keywords': ['r', 'movies']},
 '5f0tug': {'positive_keywords': ['Rocket', 'Science']},
 '5f3fwg': {'positive_keywords': ['Elle']},
 '5g3rvc': {'positive_keywords': ['Z', 'World', 'War']},
 '5g6viv': {'positive_keywords': ['Von', 'Lars', 'Trier']},
 '5g7m1z': {'positive_keywords': ['Paradiso', 'Cinema', 'Kes']},
 '5gb8uw': {'positive_keywords': ['movies']},
 '5gchx2': {'positive_keywords': ['urban', 'life']},
 '5gewuy': {'positive_keywords': ['films']},
 '5gfacz': {'positive_keywords': ['inception']},
 '5giohb': {'positive_keywords': ['90s', 'movies']},
 '5h4x1t'

In [18]:
TP_p_movies, TP_p_genres, TP_p_actors, TP_p_keywords = 0, 0, 0, 0
TP_n_movies, TP_n_genres, TP_n_actors, TP_n_keywords = 0, 0, 0, 0
FP_p_movies, FP_p_genres, FP_p_actors, FP_p_keywords = 0, 0, 0, 0
FP_n_movies, FP_n_genres, FP_n_actors, FP_n_keywords = 0, 0, 0, 0
FN_p_movies, FN_p_genres, FN_p_actors, FN_p_keywords = 0, 0, 0, 0
FN_n_movies, FN_n_genres, FN_n_actors, FN_n_keywords = 0, 0, 0, 0

results = dict()
for key, value in all_predictions.items():
    TP_p_movies_sep, TP_p_genres_sep, TP_p_actors_sep, TP_p_keywords_sep = 0, 0, 0, 0
    TP_n_movies_sep, TP_n_genres_sep, TP_n_actors_sep, TP_n_keywords_sep = 0, 0, 0, 0
    FP_p_movies_sep, FP_p_genres_sep, FP_p_actors_sep, FP_p_keywords_sep = 0, 0, 0, 0
    FP_n_movies_sep, FP_n_genres_sep, FP_n_actors_sep, FP_n_keywords_sep = 0, 0, 0, 0
    
    for cat in value.keys():
        if cat=='positive_genres':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_p_genres+=1
                    FP_p_genres_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_p_genres+=1
                        TP_p_genres_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_p_genres+=1
                        FP_p_genres_sep+=1
        if cat=='negative_genres':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_n_genres+=1
                    FP_n_genres_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_n_genres+=1
                        TP_n_genres_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_n_genres+=1  
                        FP_n_genres_sep+=1
                    
        if cat=='negative_actors':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_n_actors+=1
                    FP_n_actors_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_n_actors+=1
                        TP_n_actors_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_n_actors+=1
                        FP_n_actors_sep+=1
                    
        if cat=='positive_actors':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_p_actors+=1
                    FP_p_actors_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_p_actors+=1
                        TP_p_actors_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_p_actors+=1
                        FP_p_actors_sep+=1
            
        if cat=='positive_keywords':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_p_keywords+=1
                    FP_p_keywords_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_p_keywords+=1
                        TP_p_keywords_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_p_keywords+=1
                        FP_p_keywords_sep+=1
 
                    
        if cat=='negative_keywords':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_n_keywords+=1
                    FP_n_keywords_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_n_keywords+=1
                        TP_n_keywords_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_n_keywords+=1
                        FP_n_keywords_sep+=1

                    
        if cat=='negative_movies':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_n_movies+=1
                    FP_n_movies_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_n_movies+=1
                        TP_n_movies_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_n_movies+=1
                        FP_n_movies_sep+=1

                    
        if cat=='positive_movies':
            for item in value[cat]:
                if cat not in test_submissions[key].keys():
                    FP_p_movies+=1
                    FP_p_movies_sep+=1
                elif cat in test_submissions[key].keys():
                    if item in test_submissions[key][cat]:
                        TP_p_movies+=1
                        TP_p_movies_sep+=1
                    elif item not in test_submissions[key][cat]:
                        FP_p_movies+=1
                        FP_p_movies_sep+=1
            
    results[key] = dict()
    results[key]['positive_movies'] = {"FP" : FP_p_movies_sep, "TP" : TP_p_movies_sep}
    results[key]['negative_movies'] = {"FP" : FP_n_movies_sep, "TP" : TP_n_movies_sep}
    results[key]['positive_genres'] = {"FP" : FP_p_genres_sep, "TP" : TP_p_genres_sep}
    results[key]['negative_genres'] = {"FP" : FP_n_genres_sep, "TP" : TP_n_genres_sep}
    results[key]['positive_keywords'] = {"FP" : FP_p_keywords_sep, "TP" : TP_p_keywords_sep}
    results[key]['negative_keywords'] = {"FP" : FP_n_keywords_sep, "TP" : TP_n_keywords_sep}
    results[key]['positive_actors'] = {"FP" : FP_p_actors_sep, "TP" : TP_p_actors_sep}
    results[key]['negative_actors'] = {"FP" : FP_n_actors_sep, "TP" : TP_n_actors_sep}
    
for key, value in test_submissions.items():
    FN_p_movies_sep, FN_p_genres_sep, FN_p_actors_sep, FN_p_keywords_sep = 0, 0, 0, 0
    FN_n_movies_sep, FN_n_genres_sep, FN_n_actors_sep, FN_n_keywords_sep = 0, 0, 0, 0
    
    for cat in value.keys():
        if cat=='positive_genres':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_p_genres+=1
                    FN_p_genres_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_p_genres+=1
                    FN_p_genres_sep+=1
        if cat=='negative_genres':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_n_genres+=1
                    FN_n_genres_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_n_genres+=1
                    FN_n_genres_sep+=1
        if cat=='positive_movies':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_p_movies+=1
                    FN_p_movies_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_p_movies+=1
                    FN_p_movies_sep+=1
        if cat=='negative_movies':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_n_movies+=1
                    FN_n_movies_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_n_movies+=1
                    FN_n_movies_sep+=1
        if cat=='positive_keywords':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_p_keywords+=1
                    FN_p_keywords_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_p_keywords+=1
                    FN_p_keywords_sep+=1
        if cat=='negative_keywords':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_n_keywords+=1
                    FN_n_keywords_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_n_keywords+=1
                    FN_n_keywords_sep+=1
        if cat=='positive_actors':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_p_actors+=1
                    FN_p_actors_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_p_actors+=1
                    FN_p_actors_sep+=1
        if cat=='negative_actors':
            for item in value[cat]:
                if cat not in all_predictions_matched[key].keys():
                    FN_n_actors+=1
                    FN_n_actors_sep+=1
                elif item not in all_predictions_matched[key][cat]:
                    FN_n_actors+=1
                    FN_n_actors_sep+=1
                    
    results[key]['positive_movies'].update({"FN" : FN_p_movies_sep})
    results[key]['negative_movies'].update({"FN" : FN_n_movies_sep})
    results[key]['positive_genres'].update({"FN" : FN_p_genres_sep})
    results[key]['negative_genres'].update({"FN" : FN_n_genres_sep})
    results[key]['positive_actors'].update({"FN" : FN_p_actors_sep})
    results[key]['negative_actors'].update({"FN" : FN_n_actors_sep})
    results[key]['positive_keywords'].update({"FN" : FN_p_keywords_sep})
    results[key]['negative_keywords'].update({"FN" : FN_n_keywords_sep})

NameError: name 'test_submissions' is not defined